In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.width', 500)

In [2]:
df_ = pd.read_excel("online_retail_II.xlsx",  sheet_name="Year 2009-2010")

In [20]:
df = df_.copy()

In [21]:
def datareview(dataframe):
    print("******head******")
    print(dataframe.head(10))
    print("******shape******")
    print(dataframe.shape)
    print("******info********")
    print(dataframe.info())
    print("******describe********")
    print(dataframe.describe().T)
    print("***** Nan data********")
    print(dataframe.isnull().sum())

In [22]:
datareview(df)

******head******
  Invoice StockCode                          Description  Quantity         InvoiceDate  Price  Customer ID         Country
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12 2009-12-01 07:45:00   6.95     13085.00  United Kingdom
1  489434    79323P                   PINK CHERRY LIGHTS        12 2009-12-01 07:45:00   6.75     13085.00  United Kingdom
2  489434    79323W                  WHITE CHERRY LIGHTS        12 2009-12-01 07:45:00   6.75     13085.00  United Kingdom
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48 2009-12-01 07:45:00   2.10     13085.00  United Kingdom
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24 2009-12-01 07:45:00   1.25     13085.00  United Kingdom
5  489434     22064           PINK DOUGHNUT TRINKET POT         24 2009-12-01 07:45:00   1.65     13085.00  United Kingdom
6  489434     21871                  SAVE THE PLANET MUG        24 2009-12-01 07:45:00   1.25     13085.00  United Kingdom

In [23]:
df.dropna(inplace=True)

In [24]:
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Price"] > 0]
df = df[df["Quantity"] > 0]
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [25]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.20)
    quartile3 = dataframe[variable].quantile(0.80)
    interquantile_range = quartile3 - quartile1
    up_limit = round(quartile3 + 1.5 * interquantile_range)
    low_limit = round(quartile1 - 1.5 * interquantile_range)
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    #dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [26]:
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

In [27]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,407664.00,8.05,8.03,1.00,2.00,5.00,12.00,27.00
Price,407664.00,2.80,2.35,0.00,1.25,1.95,3.75,9.00
Customer ID,407664.00,15368.59,1679.76,12346.00,13997.00,15321.00,16812.00,18287.00
TotalPrice,407664.00,21.66,77.15,0.00,4.95,11.90,19.50,15818.40


In [28]:
dataframe = df.copy()

In [29]:
dataframe = df.groupby("Customer ID").agg({'TotalPrice':"sum"})

In [34]:
dataframe.head()

,TotalPrice
Customer ID,
18102.00,349164.35
14646.00,248396.50
14156.00,196566.74
14911.00,152147.57
13694.00,131443.19


In [35]:
dataframe = dataframe.sort_values('TotalPrice', ascending=False)

In [36]:
dataframe.head()

,TotalPrice
Customer ID,
18102.00,349164.35
14646.00,248396.50
14156.00,196566.74
14911.00,152147.57
13694.00,131443.19


In [37]:
dataframe.reset_index(inplace=True)

In [39]:
dataframe.head()

,Customer ID,TotalPrice
0,18102.00,349164.35
1,14646.00,248396.50
2,14156.00,196566.74
3,14911.00,152147.57
4,13694.00,131443.19


In [42]:
dataframe['CumSum'] = dataframe['TotalPrice'].cumsum()

In [43]:
dataframe.head()

,Customer ID,TotalPrice,CumSum
0,18102.00,349164.35,349164.35
1,14646.00,248396.50,597560.85
2,14156.00,196566.74,794127.59
3,14911.00,152147.57,946275.16
4,13694.00,131443.19,1077718.35


In [44]:
threshold = dataframe['TotalPrice'].sum() * 0.8

In [45]:
threshold

7065602.619199999

In [46]:
target_df = dataframe[dataframe['CumSum'] <= threshold]

In [48]:
target_df.head()

,Customer ID,TotalPrice,CumSum
0,18102.00,349164.35,349164.35
1,14646.00,248396.50,597560.85
2,14156.00,196566.74,794127.59
3,14911.00,152147.57,946275.16
4,13694.00,131443.19,1077718.35


In [49]:
target_df.shape

(1175, 3)

In [50]:
round(target_df.shape[0] / dataframe.shape[0],2)

0.27

In [51]:
def pareto_analysis(dataframe, id_, price_col, percentile=0.8):
    dataframe = dataframe.groupby(id_).agg({price_col: "sum"})
    dataframe = dataframe.sort_values(price_col, ascending=False)
    dataframe.reset_index(inplace=True)
    dataframe['CumSum'] = dataframe[price_col].cumsum()
    threshold = dataframe[price_col].sum() * percentile
    target_df = dataframe[dataframe['CumSum'] <= threshold]
    print("Toplam Kazanç:", dataframe[price_col].sum())
    print(f"Toplam kazancın %{100 * percentile} kısmı", target_df.shape[0], "kullanıcıdan gelmekte.")
    print(f"Toplam kazancın %{100 * percentile} 'ini getiren kullanıcılar, tüm kitlenin % {round((target_df.shape[0] * 100 / dataframe.shape[0]),2)} kesimini oluşturmakta.")



In [52]:
pareto_analysis(df, "Customer ID", 'TotalPrice', percentile=0.75)

Toplam Kazanç: 8832003.273999998
Toplam kazancın %75.0 kısmı 927 kullanıcıdan gelmekte.
Toplam kazancın %75.0 'ini getiren kullanıcılar, tüm kitlenin % 21.5 kesimini oluşturmakta.
